# Rethinking Statistics course in Stan - Week 7

Lecture 13: Monsters & Mixtures (Poisson GLMs, survival, zero-inflation)

- [Video](https://www.youtube.com/watch?v=p7g-CgGCS34)
- [Slides](https://speakerdeck.com/rmcelreath/l13-statistical-rethinking-winter-2019)

Lecture 14: Ordered Categories, Left & Right

- [Video](https://www.youtube.com/watch?v=zA3Jxv8LOrA)
- [Slides](https://speakerdeck.com/rmcelreath/l14-statistical-rethinking-winter-2019)

[Proposed problems](https://github.com/gbosquechacon/statrethinking_winter2019/blob/master/homework/week07.pdf) and [solutions in R](https://github.com/gbosquechacon/statrethinking_winter2019/blob/master/homework/week07_solutions.pdf) for the exercises of the week.

In [1]:
import pandas as pd
import numpy as np
from scipy.special import expit
from cmdstanpy import CmdStanModel
from plotnine import *

In [2]:
%load_ext watermark
%watermark -n -u -p pandas,numpy,cmdstanpy,plotnine

Last updated: Fri Dec 31 2021

pandas   : 1.3.4
numpy    : 1.21.4
cmdstanpy: 1.0.0
plotnine : 0.8.0



## Exercise 1

> In the Trolley data, `Trolley`, we saw how education level (modeled as an ordered category) is associated with responses. Is this association causal? One plausible confound is that education is also associated with age, through a causal process: People are older when they finish school than when they begin it.

> Reconsider the `Trolley` data in this light. Draw a DAG that represents hypothetical causal relationships among response, education, and age. Which statical model or models do you need to evaluate the causal influence of education on responses? Fit these models to the trolley data. What do you conclude about the causal relationships among these three variables?

This is my DAG:

<img src="./fig/w7_img1.png" width="30%">

Age could influence both response `R` and education `E`. It could influence `R`, because people at different ages could have different attitudes. Age could influence education, because the longer you have lived, the more education you could have completed (up to a point). It's like the age causing marriage example from earlier in the course. To evaluate the causal influence of `E` on `R`, we need to block the back-door from `E` through `A` to `R`. So we need a model that conditions on both `E` and `A`. Then the estimate for `E` should be the causal influence of `E`.

In [3]:
d = pd.read_csv('./dat/Trolley.csv', header=0, sep=';')
elvl = d['edu'].unique()
idx = [7 , 0 , 6 , 4 , 2 , 1, 3, 5]
cat = {elvl[i]:i for i in idx}
d['edu_cat'] = d.edu.replace(cat)
d['age_std'] = (d.age - d.age.mean())/d.age.std()
d.tail(3)

,case,response,order,id,age,male,edu,action,intention,contact,story,action2,edu_cat,age_std
9927,ilshi,7,7,98;299,66,1,Graduate Degree,0,1,0,shi,0,5,2.003041
9928,ilswi,2,18,98;299,66,1,Graduate Degree,0,1,0,swi,0,5,2.003041
9929,nfrub,2,17,98;299,66,1,Graduate Degree,1,0,0,rub,1,5,2.003041


In [4]:
model = '''

data {
    int n;
    int k;
    int edu_cat[n];
    vector[n] age_std;
    int response[n];
}

parameters {
    ordered[k] cutpoints;
    real bAge;
    real bEdu;
}

model {
    // prior
    cutpoints ~ normal(0,15);
    bAge ~ normal(0,0.5);
    bEdu ~ normal(0,0.5);
    // likelihood
    vector[n] phi;
    for (i in 1:n) {
        phi[i] = bAge * age_std[i] + bEdu * edu_cat[i];
        response[i] ~ ordered_logistic(phi[i], cutpoints);
    }
}

'''

stan_file = './stn/week07_01.stan'
with open(stan_file, 'w') as f:
    print(model, file=f)
    
model = CmdStanModel(stan_file=stan_file)
model.compile()

INFO:cmdstanpy:compiling stan file /home/jovyan/work/statret/stn/week07_01.stan to exe file /home/jovyan/work/statret/stn/week07_01
INFO:cmdstanpy:compiled model executable: /home/jovyan/work/statret/stn/week07_01
INFO:cmdstanpy:found newer exe file, not recompiling


In [5]:
%%time
data = d[['edu_cat', 'age_std', 'response']].copy()
data = data.to_dict(orient='list')
data['n'] = len(data['response'])
data['k'] = d.response.nunique()-1
fit = model.sample(data=data, chains=4)

INFO:cmdstanpy:CmdStan start procesing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

INFO:cmdstanpy:CmdStan done processing.



CPU times: user 354 ms, sys: 183 ms, total: 537 ms
Wall time: 2min 33s


In [6]:
fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-19000.00,0.05000,2.000,-19000.00,-19000.00,-19000.00,1600.00,5.40,1.0
cutpoints[1],-1.90,0.00063,0.040,-2.00,-1.90,-1.80,4043.00,13.00,1.0
cutpoints[2],-1.20,0.00054,0.036,-1.30,-1.20,-1.20,4419.00,15.00,1.0
cutpoints[3],-0.69,0.00050,0.034,-0.75,-0.69,-0.63,4788.00,16.00,1.0
cutpoints[4],0.28,0.00049,0.034,0.22,0.28,0.33,4900.00,16.00,1.0
cutpoints[5],0.92,0.00051,0.035,0.86,0.92,0.98,4752.00,16.00,1.0
cutpoints[6],1.80,0.00059,0.040,1.70,1.80,1.90,4646.00,15.00,1.0
bAge,-0.07,0.00000,0.020,-0.10,-0.07,-0.04,5843.48,19.27,1.0
bEdu,0.01,0.00000,0.010,-0.01,0.01,0.03,4826.93,15.91,1.0


You may recall from the chapter that education has a negative effect in the model without age. Now that we include age, education has a positive influence (with some overlap with zero). So age has indeed soaked up some of the previous influence assigned to education. The back-door may be real.

I'd summarize this model, assuming this DAG is true, as saying that age causes people to give slightly lower responses. This could be a cohort effect, and not a causal influence of age. Either way, it is small. Education seems to cause higher responses (more approval). This suggests that education trains people to see some or all of the features A, I, C as more permissible. A model that interacted education with each might shed more light on things. Remember: A DAG doesn't say whether you need an interaction effect or not. That is a separate problem.

## Exercise 2

> Consider one more variable in the Trolley data: Gender. Suppose that gender might influence education as well as response directly. Draw the DAG now that includes response, education, age, and gender.

> Using only the DAG, is it possible that the inferences from Problem 1 are confounded by gender? If so, define any additional models you need to infer the causal influence of education on response. What do you conclude?

This is my DAG:

<img src="./fig/w7_img2.png" width="30%">

Here's the model we need, which includes education, age, and gender (female dummy variable):

In [7]:
d = pd.read_csv('./dat/Trolley.csv', header=0, sep=';')
elvl = d['edu'].unique()
idx = [7 , 0 , 6 , 4 , 2 , 1, 3, 5]
cat = {elvl[i]:i for i in idx}
d['edu_cat'] = d.edu.replace(cat)
d['age_std'] = (d.age - d.age.mean())/d.age.std()
d.tail(3)

,case,response,order,id,age,male,edu,action,intention,contact,story,action2,edu_cat,age_std
9927,ilshi,7,7,98;299,66,1,Graduate Degree,0,1,0,shi,0,5,2.003041
9928,ilswi,2,18,98;299,66,1,Graduate Degree,0,1,0,swi,0,5,2.003041
9929,nfrub,2,17,98;299,66,1,Graduate Degree,1,0,0,rub,1,5,2.003041


In [8]:
model = '''

data {
    int n;
    int k;
    int edu_cat[n];
    int male[n];
    vector[n] age_std;
    int response[n];
}

parameters {
    ordered[k] cutpoints;
    real bAge;
    real bEdu;
    real bMale;
}

model {
    // prior
    cutpoints ~ normal(0,15);
    bAge ~ normal(0,0.5);
    bEdu ~ normal(0,0.5);
    // likelihood
    vector[n] edu;
    vector[n] phi;
    for (i in 1:n) {
        edu[i] = bAge * age_std[i] + bMale * male[i];
        phi[i] = bAge * age_std[i] + bMale * male[i] + bEdu * edu_cat[i];
        response[i] ~ ordered_logistic(phi[i], cutpoints);
    }
}

'''

stan_file = './stn/week07_02.stan'
with open(stan_file, 'w') as f:
    print(model, file=f)
    
model = CmdStanModel(stan_file=stan_file)
model.compile()

INFO:cmdstanpy:compiling stan file /home/jovyan/work/statret/stn/week07_02.stan to exe file /home/jovyan/work/statret/stn/week07_02
INFO:cmdstanpy:compiled model executable: /home/jovyan/work/statret/stn/week07_02
INFO:cmdstanpy:found newer exe file, not recompiling


In [9]:
%%time
data = d[['edu_cat', 'male', 'age_std', 'response']].copy()
data = data.to_dict(orient='list')
data['n'] = len(data['response'])
data['k'] = d.response.nunique()-1
fit = model.sample(data=data, chains=4)

INFO:cmdstanpy:CmdStan start procesing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

INFO:cmdstanpy:CmdStan done processing.



CPU times: user 430 ms, sys: 120 ms, total: 550 ms
Wall time: 3min 12s


In [10]:
fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-19000.00,0.05200,2.100,-19000.00,-19000.00,-19000.00,1700.00,4.50,1.0
cutpoints[1],-1.60,0.00073,0.045,-1.70,-1.60,-1.50,3741.00,10.00,1.0
cutpoints[2],-0.97,0.00063,0.041,-1.00,-0.97,-0.90,4123.00,11.00,1.0
cutpoints[3],-0.41,0.00059,0.039,-0.48,-0.41,-0.35,4471.00,12.00,1.0
cutpoints[4],0.57,0.00057,0.040,0.51,0.57,0.64,4767.00,13.00,1.0
cutpoints[5],1.20,0.00059,0.041,1.20,1.20,1.30,4889.00,13.00,1.0
cutpoints[6],2.10,0.00064,0.046,2.10,2.10,2.20,5047.00,14.00,1.0
bAge,-0.06,0.00000,0.020,-0.08,-0.06,-0.03,7561.29,20.38,1.0
bEdu,0.01,0.00000,0.010,-0.01,0.01,0.03,4961.70,13.37,1.0


Age is still negative (and weak), while education is right near zero and straddles both sides. Gender seems to have accounted for all of the previous influenced assigned to education. It looks like female respondents gave lower average responses, indicating less approval.

It would be worth figuring out how gender is associated with education in this sample. It could be true for example that some education levels under-sampled men or women, and this leads to another kind of confound. Consider for example if older men are less likely to respond, so the sample becomes increasingly female with age. Then education level will also be increasingly female with age. Since the sample is
not a representative sample of the population, there are probably some biases of this sort.